<a href="https://colab.research.google.com/github/Jawad5747/Dissertation/blob/main/fake_news_eng_thesis_Copy1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import texthero as hero
import numpy as np
import warnings
warnings.filterwarnings('ignore')

## Data insertion 

In [ ]:
df=pd.readf=pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df=df.dropna()
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [ ]:
X=df.drop(['id','title','label','author'],axis=1)
y=df['label']

## Preprocessing

In [ ]:
#preprocessing
X['clean_content'] = hero.clean(X['text'])

In [ ]:
#digit removal
X['clean_content'] = X['clean_content'].str.replace('\d+', '')

## Removing tokens whose length is less then 4

In [ ]:
#remove whose length is less then 3 
df_text=X['clean_content'].str.findall('\w{3,}').str.join(' ')

## Top words with maximum frequency 

In [ ]:
#Top words
hero.top_words(df_text)[5:10]

new          28275
like         24812
clinton      24464
also         23825
president    23736
Name: clean_content, dtype: int64

# Content based methods (TF-IDF+BOW)

In [ ]:
#content based features TFIDF and CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer as count_vect

from sklearn.feature_extraction.text import CountVectorizer as tfidf
tf_idf=tfidf(max_features=2000,ngram_range=(1,1),analyzer='word')

In [ ]:
mat = tf_idf.fit_transform(df_text)
den_mat=mat.todense()

## Train test split for content based methods

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(den_mat, y, train_size=0.75)

### Gaussian nb

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB().fit(X_train, y_train)
pred_gnb = gnb.predict(X_test)
score_gnb = metrics.accuracy_score(y_test, pred_gnb)
print(" gnb accuracy:   %0.4f" % score_gnb)

 gnb accuracy:   0.8082


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier().fit(X_train, y_train)
pred_knn = knn.predict(X_test)
score_knn = metrics.accuracy_score(y_test, pred_knn)
print(" gnb accuracy:   %0.4f" % score_knn)

 gnb accuracy:   0.7675


### Classification report

In [ ]:
print(metrics.classification_report(pred_knn,y_test))

              precision    recall  f1-score   support

           0       0.65      0.91      0.76      1807
           1       0.92      0.67      0.78      2765

    accuracy                           0.77      4572
   macro avg       0.78      0.79      0.77      4572
weighted avg       0.81      0.77      0.77      4572

